In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

%matplotlib inline

In [1]:
import sys
sys.path.append('/notebooks/carputils')
sys.path.append('/notebooks/examples')

sys.path

['',
 '/opt/conda/lib/python27.zip',
 '/opt/conda/lib/python2.7',
 '/opt/conda/lib/python2.7/plat-linux2',
 '/opt/conda/lib/python2.7/lib-tk',
 '/opt/conda/lib/python2.7/lib-old',
 '/opt/conda/lib/python2.7/lib-dynload',
 '/opt/conda/lib/python2.7/site-packages',
 '/opt/conda/lib/python2.7/site-packages/Sphinx-1.5.1-py2.7.egg',
 '/opt/conda/lib/python2.7/site-packages/IPython/extensions',
 '/root/.ipython',
 '/notebooks/carputils',
 '/notebooks/examples']

In [13]:
%run -i ../examples/tutorials/02_EP_tissue/01_basic_usage/run.py --duration 20 --S1-strength 20. --S1-dur 15

SETTINGS SUMMARY
Executables
----------------------------------------------------------------------
   CARP:            /notebooks/carputils/bin/carphelp
   MESHER:          /notebooks/carputils/bin/mesher
   FSM:             < NOT FOUND >
   BENCH:           < NOT FOUND >
   GLINTERPOLATE:   < NOT FOUND >
   GLGRADIENT:      < NOT FOUND >
   GLELEMCENTERS:   < NOT FOUND >
   GLMESHCONVERT:   < NOT FOUND >
   GLVTKCONVERT:    < NOT FOUND >
   GLTRANSFER:      < NOT FOUND >
   GLRULEFIBERS:    < NOT FOUND >
   ELLIPTICSOLVER:  < NOT FOUND >
   PARABOLICSOLVER: < NOT FOUND >
   ADJOINTSOLVER:   < NOT FOUND >
   GLFILAMENT:      < NOT FOUND >
   GLIGBPROCESS:    < NOT FOUND >
   SURFACEMESH:     < NOT FOUND >
   GLSURFACECONVERT: < NOT FOUND >
   MESHTOOL:        < NOT FOUND >
   ELASTICITY:      < NOT FOUND >
   FLUIDSOLVE:      < NOT FOUND >
   CVSTOOL:         < NOT FOUND >
   IGBHEAD:         < NOT FOUND >
   IGBOPS:          < NOT FOUND >
   IGBEXTRACT:      < NOT FOUND >
   IGBAPD: 

In [19]:
! which python

/opt/conda/bin/python
